# 数据分组 Groupby

数据有关键词、每个数据带有特定类别？我们怎样可以快速找到对应的类别，聚类分析？

Pandas提供了groupby()函数，可以对数据进行分组，并对分组后的结果进行操作。

### 分组
对于 groupby() 最核心的功能，自然就是将可以被归纳的数据进行归纳汇总。

In [1]:
import pandas as pd

df = pd.DataFrame(
    [
        ("小红", "哈利波特", 80),
        ("小明", "蜘蛛侠", 72),
        ("小红", "雷神", 83),
        ("小红", "蜘蛛侠", 45),
        ("小明", "超人", 57),
    ],
    columns=("人", "人物", "评价"),
)
df


,人,人物,评价
0,小红,哈利波特,80
1,小明,蜘蛛侠,72
2,小红,雷神,83
3,小红,蜘蛛侠,45
4,小明,超人,57


In [3]:
grouped = df.groupby("人")
grouped.groups

{'小明': [1, 4], '小红': [0, 2, 3]}

你会发现，它保存的其实是不同 人 的 row index, 这样后面重新在找 group 组的时候，可以快速用 index 找到对应行。 

要直接获取不同的 人 组，我们看看怎么搞？

In [4]:
df.iloc[grouped.groups["小红"]]

,人,人物,评价
0,小红,哈利波特,80
2,小红,雷神,83
3,小红,蜘蛛侠,45


In [6]:
# 另一种方法

grouped.get_group("小红")


,人,人物,评价
0,小红,哈利波特,80
2,小红,雷神,83
3,小红,蜘蛛侠,45


### 调用分好的组

In [7]:
grouped.first()

,人物,评价
人,,
小明,蜘蛛侠,72
小红,哈利波特,80


In [8]:
grouped.last()

,人物,评价
人,,
小明,超人,57
小红,蜘蛛侠,45


In [9]:
grouped.sum()

C:\Users\Style丶深情\AppData\Local\Temp\ipykernel_4404\2368404282.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped.sum()


,评价
人,
小明,129
小红,208


### 循环处理

In [10]:
for name, group in grouped:
    print("name:", name)
    print(group)


name: 小明
    人   人物  评价
1  小明  蜘蛛侠  72
4  小明   超人  57
name: 小红
    人    人物  评价
0  小红  哈利波特  80
2  小红    雷神  83
3  小红   蜘蛛侠  45


### 多从分组

In [11]:
df = pd.DataFrame(
    [
        ("小红", "哈利波特", 80),
        ("小明", "蜘蛛侠", 72),
        ("小红", "雷神", 83),
        ("小红", "雷神", 90),
        ("小红", "蜘蛛侠", 45),
        ("小明", "超人", 57),
    ],
    columns=("人", "人物", "评价"),
)
df

,人,人物,评价
0,小红,哈利波特,80
1,小明,蜘蛛侠,72
2,小红,雷神,83
3,小红,雷神,90
4,小红,蜘蛛侠,45
5,小明,超人,57


In [12]:
df.groupby(["人", "人物"]).get_group(("小红", "雷神"))

,人,人物,评价
2,小红,雷神,83
3,小红,雷神,90


### 聚合计算

In [13]:
import numpy as np

grouped = df.groupby("人")
grouped.aggregate(np.sum)


C:\Users\Style丶深情\AppData\Local\Temp\ipykernel_4404\3210647702.py:4: FutureWarning: The operation <function sum at 0x00000147BE0FB010> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  grouped.aggregate(np.sum)


,评价
人,
小明,129
小红,298


如果你想要对数据一次性做更多的计算，你可以多加几个要计算的操作在后面，比如 [sum, mean, std]

注意下面的 .agg() 其实是 .aggregate() 的缩写， 它俩是一样的。

In [14]:
grouped["评价"].agg([np.sum, np.mean, np.std])

,sum,mean,std
人,,,
小明,129,64.5,10.606602
小红,298,74.5,20.108042


In [15]:
grouped["评价"].agg(
    [np.sum, np.mean, np.std]
).rename(columns={
    "sum": "合", 
    "mean": "均值", 
    "std": "标准差"
})

,合,均值,标准差
人,,,
小明,129,64.5,10.606602
小红,298,74.5,20.108042
